# LLM Utility Package

This package is a simple wrapper for the main LLM providers with the minimal viable set of feature supports (for my workflow). It supports:

1. Concurrent text embeddings (except for Claude)
2. Saving to a chromadb vector store
2. Concurrent chat calls
3. Web search
4. Structured outputs

## 1. Environment Setup

Before running the code, ensure you have a `.env` file in your project root with the following keys:

```bash
GEMINI_TYPE=
GEMINI_PROJECT_ID=
GEMINI_PRIVATE_KEY_ID=
GEMINI_PRIVATE_KEY=
GEMINI_CLIENT_EMAIL=
GEMINI_CLIENT_ID=
GEMINI_AUTH_URI=
GEMINI_TOKEN_URI=
GEMINI_AUTH_PROVIDER_CERT=
GEMINI_CLIENT_CERT_URL=
GEMINI_UNIVERSE_DOMAIN=
ANTHROPIC_API_KEY=
OPENAI_KEY=

```

In [17]:
from dotenv import load_dotenv
import os
import numpy as np
import json

# Load environment variables
load_dotenv()

True

## 2. Initialization

Import and initialize the `LLMWrapper`. This class handles routing requests to the appropriate provider based on the model name.

In [18]:
from llm_utils.wrapper import LLMWrapper

llm = LLMWrapper()

## 3. Basic Chat Completion

The `ask` method is the main entry point. It takes lists of inputs (ids, messages, formats) to concurrent processing, but here we'll show single requests.

In [19]:
system_message = "You are a helpful assistant."
user_messages = [
    "Briefly explain the theory of relativity like I'm five. Then explain it again like I'm a college student.",
    "What is the current price of Tesla's stock? If you can find it, return N/A"
]
response_formats = [
    {
        "name": "example_response1",
        "type": "json_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "FIVE_YEAR_OLD_EXPLANATION": {"type": "string"},
                "COLLEGE_STUDENT_EXPLANATION": {"type": "string"}
            },
            "required": ["FIVE_YEAR_OLD_EXPLANATION", "COLLEGE_STUDENT_EXPLANATION"],
            "additionalProperties": False
        }
    },
    {
        "name": "example_response2",
        "type": "json_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "TICKER": {"type": "string"},
                "CURRENT_PRICE": {"type": "string"},
                "EXCHANGE": {"type": "string"}
            },
            "required": ["TICKER", "CURRENT_PRICE", "EXCHANGE"],
            "additionalProperties": False
        }
    }
]

### OpenAI

In [24]:
response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gpt-5-mini",
    n_workers=2,
)

display(response)

response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gpt-5-mini",
    web_search=True,
    n_workers=2,
)

display(response)

Chat Completion: 100%|██████████| 2/2 [00:16<00:00,  8.37s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': 'Think of space and time like a big stretchy trampoline. When something heavy, like the Sun, sits on the trampoline it makes a dip, and smaller things like planets roll around it — that’s what we call gravity. Also, there’s a fastest thing (light), and if you go really, really fast, your clock would tick slower and your ruler would look shorter — but only when you’re moving super fast, not in everyday life.',
  'COLLEGE_STUDENT_EXPLANATION': "The theory of relativity comprises special relativity and general relativity. Special relativity (Einstein, 1905) is based on two postulates: the laws of physics are the same in all inertial frames, and the speed of light c is constant in every inertial frame. From these follow the Lorentz transformations and effects like time dilation (t' = γ t), length contraction, and the relativity of simultaneity, with γ = 1/√(1−v²/c²). The spacetime interval Δs² = −c²Δt² + Δx² + Δy² + Δz² is invariant between inertial obse

Chat Completion: 100%|██████████| 2/2 [00:21<00:00, 10.74s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': 'Imagine space and time are like a big stretchy trampoline. Heavy things (like planets or stars) make dents in the trampoline, and smaller things roll toward the dents — that’s what we feel as gravity. Also, if you run super, super fast or sit next to a very heavy thing, your clock will tick a little differently than someone far away — time can stretch or squish. Light is the fastest thing and everyone measures it the same speed, so moving doesn’t make light go faster for you. That’s the simple idea of relativity: big things bend the trampoline of space-time, and motion or gravity can change how time and distance look.',
  'COLLEGE_STUDENT_EXPLANATION': "Relativity comes in two parts: special relativity (SR) and general relativity (GR). SR (Einstein, 1905) is based on two postulates: the laws of physics are the same in all inertial frames, and the speed of light c is constant for all inertial observers. From these follow the Lorentz transformations: 

### Gemini (Gemini 2.5 Flash)

In [25]:
response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gemini-2.5-flash",
    n_workers=2,
)

display(response)

response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gemini-2.5-flash",
    web_search=True,
    n_workers=2,
)

display(response)

Chat Completion: 100%|██████████| 2/2 [00:06<00:00,  3.00s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': "Imagine you're playing your favorite game, and time goes super fast, right? But when you have to wait for something boring, time goes super slow! Grown-ups found out that time can *really* go faster or slower depending on how fast you're moving or how heavy things are around you, like big, big stars! It's a bit like time being stretchy.",
  'COLLEGE_STUDENT_EXPLANATION': 'The theory of relativity, proposed by Albert Einstein, comprises two interconnected theories: special relativity (1905) and general relativity (1915). Special relativity addresses the relationship between space and time for objects moving at constant speeds relative to each other. Its core postulates are the constancy of the speed of light for all inertial observers and the invariance of physical laws in all inertial frames. This leads to phenomena like time dilation, length contraction, and mass-energy equivalence (E=mc²). General relativity extends this to include gravity, reinte

Chat Completion: 100%|██████████| 2/2 [00:14<00:00,  7.02s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': 'Imagine you\'re playing with your toy car, and it goes super, super fast – almost as fast as a zooming rocket! Something really strange happens then. If you were watching someone else drive their toy car that fast, their car might look a little squished, and their clock might tick a tiny bit slower than yours. That\'s because when things move really, really fast, space and time get a little mixed up!Also, think about a big, heavy bowling ball you put on a trampoline. It makes a dip, right? If you roll a little marble near the bowling ball, the marble rolls towards the big ball because of that dip. That\'s kind of like how really heavy things, like planets and stars, bend the "space fabric" around them, and that\'s what makes things fall down!',
  'COLLEGE_STUDENT_EXPLANATION': 'Albert Einstein\'s theory of relativity revolutionized our understanding of space, time, gravity, and the universe, superseding Newtonian mechanics. It\'s primarily composed 

### Claude (Claude 3.5 Sonnet)

In [26]:
response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="claude-sonnet-4-5",
    n_workers=2,
)

display(response)

response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="claude-sonnet-4-5",
    web_search=True,
    n_workers=2,
)

display(response)

Chat Completion: 100%|██████████| 2/2 [00:09<00:00,  4.82s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': "Imagine you're on a super fast train with your toy ball. If you bounce the ball, it looks normal to you. But to your friend standing outside watching the train zoom by, the ball looks like it's moving really, really fast! That's because things can look different depending on how fast you're moving. Also, time is like a stretchy rubber band - it can go slower or faster depending on how fast you're going. And really heavy things, like the Earth, are so heavy they make space bend like when you sit on a trampoline, and that's what we call gravity!",
  'COLLEGE_STUDENT_EXPLANATION': "Einstein's theory of relativity consists of two parts: Special Relativity (1905) and General Relativity (1915). Special Relativity deals with objects moving at constant velocities, particularly near the speed of light. It establishes that the laws of physics are the same in all inertial reference frames, and that the speed of light in vacuum is constant for all observers. Th

Chat Completion: 100%|██████████| 2/2 [00:13<00:00,  6.76s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': "Imagine you're on a super fast train with your friend. If you throw a ball to your friend, it looks normal to you. But to someone standing outside watching the train zoom by, the ball looks like it's moving really, really fast! That's because things can look different depending on where you are and how fast you're moving.\n\nAlso, time is kind of like a stretchy rubber band. If you could go really, really fast (like as fast as light), time would slow down for you! So if you went on a super fast spaceship, when you came back, you might only be a little older, but everyone on Earth would be much older. And really heavy things, like planets, make space bend like when you put a bowling ball on a trampoline - that's what makes gravity!",
  'COLLEGE_STUDENT_EXPLANATION': "Einstein's theory of relativity consists of two parts: special relativity (1905) and general relativity (1915).\n\nSpecial relativity deals with objects moving at constant velocities, pa

## Embedding

In [ ]:
texts = ["cat", "kitty", "dog", "potato"]
ids = ["1", "2", "3", "4"]

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# OpenAI Embeddings
print("Getting OpenAI Embeddings...")
_, openai_embeddings = llm.embed(
    ids=ids,
    texts=texts,
    size=1536,
    db=None,
    verbose=False,
    model="text-embedding-3-large"
)

cat_emb = np.array(openai_embeddings[0])
kitty_emb = np.array(openai_embeddings[1])
dog_emb = np.array(openai_embeddings[2])
potato_emb = np.array(openai_embeddings[3])

sim_cat_kitty = cosine_similarity(cat_emb, kitty_emb)
sim_dog_potato = cosine_similarity(dog_emb, potato_emb)

print(f"OpenAI Similarity (cat, kitty): {sim_cat_kitty:.4f}")
print(f"OpenAI Similarity (dog, potato): {sim_dog_potato:.4f}")

if sim_cat_kitty > sim_dog_potato:
    print("✅ OpenAI Embedding Test Passed: cat-kitty similarity > dog-potato similarity")
else:
    print("❌ OpenAI Embedding Test Failed")

# Gemini Embeddings
print("Getting Gemini Embeddings...")
_, gemini_embeddings = llm.embed(
    ids=ids,
    texts=texts,
    size=768, 
    db=None,
    name="test_gemini_emb",
    verbose=False,
    model="text-embedding-004"
)

cat_emb = np.array(gemini_embeddings[0])
kitty_emb = np.array(gemini_embeddings[1])
dog_emb = np.array(gemini_embeddings[2])
potato_emb = np.array(gemini_embeddings[3])

sim_cat_kitty = cosine_similarity(cat_emb, kitty_emb)
sim_dog_potato = cosine_similarity(dog_emb, potato_emb)

print(f"Gemini Similarity (cat, kitty): {sim_cat_kitty:.4f}")
print(f"Gemini Similarity (dog, potato): {sim_dog_potato:.4f}")

if sim_cat_kitty > sim_dog_potato:
    print("✅ Gemini Embedding Test Passed: cat-kitty similarity > dog-potato similarity")
else:
    print("❌ Gemini Embedding Test Failed")

Getting OpenAI Embeddings...
OpenAI Similarity (cat, kitty): 0.6139
OpenAI Similarity (dog, potato): 0.3897
✅ OpenAI Embedding Test Passed: cat-kitty similarity > dog-potato similarity
Getting Gemini Embeddings...
Gemini Similarity (cat, kitty): 0.9019
Gemini Similarity (dog, potato): 0.8507
✅ Gemini Embedding Test Passed: cat-kitty similarity > dog-potato similarity
